# LangChain

LangChain is a framework for developing applications powered by large language
models (LLMs). Currently, Memgraph's LangChain integration supports
creating a knowledge graph from unstructured data and querying with natural
language. You can follow the example on [LangChain
docs](https://python.langchain.com/docs/integrations/graphs/memgraph/) or go
through quick start below.

## Installation

To install all the required packages, run:

```shell
pip install langchain langchain-openai neo4j --user
```

## Environment setup 

Before you get started, make sure you have [Memgraph](/getting-started) running
in the background.

Then, instantiate `MemgraphGraph` in your Python code. This object holds the
connection to the running Memgraph instance. Make sure to set up all the
environment variables properly.

In [ ]:
import os

from langchain_community.chains.graph_qa.memgraph import MemgraphQAChain
from langchain_community.graphs import MemgraphGraph
from langchain_openai import ChatOpenAI

url = os.environ.get("MEMGRAPH_URI", "bolt://localhost:7687")
username = os.environ.get("MEMGRAPH_USERNAME", "")
password = os.environ.get("MEMGRAPH_PASSWORD", "")

graph = MemgraphGraph(
    url=url, username=username, password=password, refresh_schema=False
)


The `refresh_schema` is initially set to `False` because there is still no data in
the database and we want to avoid unnecessary database calls.

To interact with the LLM, you must configure it. Here is how you can set API key as an
environment variable for OpenAI:

In [ ]:
os.environ["OPENAI_API_KEY"] = "your-key-here"

## Graph construction

For the dataset, we'll use the following text about Charles Darwin:

In [ ]:
text = """
    Charles Robert Darwin was an English naturalist, geologist, and biologist,
    widely known for his contributions to evolutionary biology. His proposition that
    all species of life have descended from a common ancestor is now generally
    accepted and considered a fundamental scientific concept. In a joint
    publication with Alfred Russel Wallace, he introduced his scientific theory that
    this branching pattern of evolution resulted from a process he called natural
    selection, in which the struggle for existence has a similar effect to the
    artificial selection involved in selective breeding. Darwin has been
    described as one of the most influential figures in human history and was
    honoured by burial in Westminster Abbey.
"""

To construct the graph, first initialize `LLMGraphTransformer` from the desired
LLM and convert the document to the graph structure.

In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")
llm_transformer = LLMGraphTransformer(llm=llm)
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

The graph structure in the `GraphDocument` format can be forwarded to the
`add_graph_documents()` procedure to import in into Memgraph:

In [ ]:
# Make sure the database is empty
graph.query("STORAGE MODE IN_MEMORY_ANALYTICAL")
graph.query("DROP GRAPH")
graph.query("STORAGE MODE IN_MEMORY_TRANSACTIONAL")

# Create KG
graph.add_graph_documents(graph_documents)

The `add_graph_documents()` procedure transforms the list of `graph_documents`
into appropriate Cypher queries and executes them in Memgraph.

In the below image, you can see how the text was transformed into a knowledge
graph and stored into Memgraph.

![langchain-kg](langchain-kg-creation.png)

For additional options, check the [full
guide](https://python.langchain.com/docs/integrations/graphs/memgraph/#additional-options)
on the LangChain docs. 